In [ ]:
import os
import h5py
import numpy as np

def load_multiple_h5_files(input_dir):
    # Initialize lists to hold data from each file
    positions_list = []
    metadata_list = []
    scores_list = []

    # Iterate through each file in the directory
    for chunk_file in sorted(os.listdir(input_dir)):
        if chunk_file.endswith(".h5"):
            chunk_path = os.path.join(input_dir, chunk_file)
            with h5py.File(chunk_path, "r") as chunk_h5:
                # Append datasets from each chunk file
                positions_list.append(chunk_h5["positions"][:])
                metadata_list.append(chunk_h5["metadata"][:])
                scores_list.append(chunk_h5["scores"][:])

    # Concatenate all the data along the first axis (rows)
    positions = np.concatenate(positions_list, axis=0)
    metadata = np.concatenate(metadata_list, axis=0)
    scores = np.concatenate(scores_list, axis=0)

    return positions, metadata, scores

# Example usage
input_dir = "processed_data/train"  # Directory where your chunked .h5 files are stored
positions, metadata, scores = load_multiple_h5_files(input_dir)

# Now positions, metadata, and scores hold the combined data from all the .h5 files
print(f"Loaded {len(positions)} positions, {len(metadata)} metadata, and {len(scores)} scores.")


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from chess_dataset import ChessDataset
from get_data import load_multiple_npz
from chess_nn import ChessCNN
model = ChessCNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
criterion = nn.MSELoss()
positions, metadata, scores = load_multiple_npz("processed_data2")
dataset = ChessDataset(positions, metadata, scores)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

print(f"Using device: {device}")

Renamed: processed_data/77.npz -> processed_data/77_train.npz
Renamed: processed_data/189.npz -> processed_data/189_train.npz
Renamed: processed_data/63.npz -> processed_data/63_train.npz
Renamed: processed_data/88.npz -> processed_data/88_train.npz
Renamed: processed_data/176.npz -> processed_data/176_train.npz
Renamed: processed_data/162.npz -> processed_data/162_train.npz
Renamed: processed_data/228.npz -> processed_data/228_train.npz
Renamed: processed_data/200.npz -> processed_data/200_train.npz
Renamed: processed_data/214.npz -> processed_data/214_train.npz
Renamed: processed_data/215.npz -> processed_data/215_train.npz
Renamed: processed_data/201.npz -> processed_data/201_train.npz
Renamed: processed_data/229.npz -> processed_data/229_train.npz
Renamed: processed_data/163.npz -> processed_data/163_train.npz
Renamed: processed_data/89.npz -> processed_data/89_train.npz
Renamed: processed_data/177.npz -> processed_data/177_train.npz
Renamed: processed_data/62.npz -> processed_data

In [ ]:
for epoch in range(100):
  model.train()
  total_loss = 0.0
  for boards, additional_features, labels in data_loader:
    boards = boards.to(device, dtype=torch.float32)
    additional_features = additional_features.to(device, dtype=torch.float32)
    labels = labels.to(device, dtype=torch.float32)
    optimizer.zero_grad()
    outputs = model(boards, additional_features)

    loss = criterion(outputs.squeeze(), labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader)}")
  scheduler.step()

Using device: mps


In [ ]:
for epoch in range(100):
    model.train()
    total_loss = 0.0
    for boards, additional_features, labels in data_loader:
        boards = boards.to(device, dtype=torch.float32)
        additional_features = additional_features.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.float32)
        optimizer.zero_grad()
        outputs = model(boards, additional_features)

        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader)}")

Epoch 1, Loss: 1892695.9628474468
Epoch 2, Loss: 1725298.1848100894
Epoch 3, Loss: 1609567.8026866033
Epoch 4, Loss: 1519654.861065381
Epoch 5, Loss: 1434735.4317306562
Epoch 6, Loss: 1354379.83423242
Epoch 7, Loss: 1274174.7078672536
Epoch 8, Loss: 1201002.3969731156
Epoch 9, Loss: 1098157.7090854305
Epoch 10, Loss: 995218.4174193762
Epoch 11, Loss: 901752.8560741636
Epoch 12, Loss: 818468.890768315
Epoch 13, Loss: 726523.9548098689
Epoch 14, Loss: 670536.4730439353
Epoch 15, Loss: 602888.8990410763
Epoch 16, Loss: 546254.9877980215
Epoch 17, Loss: 507232.24130739947
Epoch 18, Loss: 467635.6194796199
Epoch 19, Loss: 435455.3507301711
Epoch 20, Loss: 414879.6375648592
Epoch 21, Loss: 389362.05451019376
Epoch 22, Loss: 367006.8994315175
Epoch 23, Loss: 345622.11513515696
Epoch 24, Loss: 328269.7202695055
Epoch 25, Loss: 323664.0727286424
Epoch 26, Loss: 311630.7147684178
Epoch 27, Loss: 303271.77136127115
Epoch 28, Loss: 290375.3228352099
Epoch 29, Loss: 266880.37360359833
Epoch 30, Los